<a href="https://colab.research.google.com/github/Mar3334/Patimetria/blob/main/Patimetria2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive #Se importa el drive donde estan todos los .py y las imagenes utilizadas, esta celda debe ser modificada en caso de que se quiera probar el codigo en otro equipo
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install torch==1.4.0 torchvision==0.5.0 #Se instala la version de torch y torchvision necesaria para poder usar el codigo.

     |████████████████████████████████| 753.4MB 20kB/s 
     |████████████████████████████████| 4.0MB 32.4MB/s 
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101


In [3]:
!cp -rf /content/drive/MyDrive/siamese-triplet /content/drive/MyDrive/siamese-triplet #Se copia en el drive cualquier cambio que se realize a los .py

cp: cannot copy a directory, '/content/drive/MyDrive/siamese-triplet', into itself, '/content/drive/MyDrive/siamese-triplet/siamese-triplet'


In [59]:
# Se importar todos los modulos y funciones necesarios
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import torch
from torch.optim import lr_scheduler
import torch.optim as optim
from torch.autograd import Variable
from torchvision import models, transforms
from PIL import Image
import sys
sys.path.append('/content/drive/MyDrive/siamese-triplet')

#La siguiente importacion es de un codigo diseñado para el proyecto patimetria:
from duckies_dataset import DuckieDataset, Rescale
#Los siguientes modulos son versiones modificadas de .py de terceros adapatados para utilizar en el proyecto patimetria:
from networks import VGGEmbeddingNet, EmbeddingNet, TripletNet
from trainer import fit, train_epoch, test_epoch
from datasets import TripletMNIST
from losses import TripletLoss

In [60]:
vgg_model = VGGEmbeddingNet()
emb_net = EmbeddingNet()

In [61]:
# Se define una transformacion que convierte las imagenes a tensores de tamaño [224, 224]
trans = transforms.Compose([Rescale(224),
                            transforms.ToTensor()])

In [62]:
# Se definen los datasets de entrenamiento y de testeo.
train_dataset = DuckieDataset("/content/drive/MyDrive/train", transform= trans)
test_dataset = DuckieDataset("/content/drive/MyDrive/train", train=False, transform= trans)

In [63]:
cuda = torch.cuda.is_available()
%matplotlib inline
# Set up data loaders

triplet_train_dataset = TripletMNIST(train_dataset) # Returns triplets of images
triplet_test_dataset = TripletMNIST(test_dataset)
batch_size = 32
kwargs = {'num_workers': 8, 'pin_memory': True} if cuda else {}
triplet_train_loader = torch.utils.data.DataLoader(triplet_train_dataset, batch_size=batch_size, shuffle=True, **kwargs)
triplet_test_loader = torch.utils.data.DataLoader(triplet_test_dataset, batch_size=batch_size, shuffle=False, **kwargs)

# Set up the network and training parameters

margin = 1.
embedding_net = EmbeddingNet()
model = TripletNet(vgg_model)
if cuda:
    model.cuda()
loss_fn = TripletLoss(margin)
lr = 1e-3
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)
n_epochs = 10
log_interval = 500

<class 'torch.Tensor'>


In [64]:
fit(triplet_train_loader, triplet_test_loader, model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval) #Se empieza a entrenar la red.

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Train: [0/1080 (0%)]	Loss: 0.830724
Epoch: 1/10. Train set: Average loss: 0.8479
Epoch: 1/10. Validation set: Average loss: 0.6013
Train: [0/1080 (0%)]	Loss: 0.598962
Epoch: 2/10. Train set: Average loss: 0.6066
Epoch: 2/10. Validation set: Average loss: 0.4509
Train: [0/1080 (0%)]	Loss: 0.527424
Epoch: 3/10. Train set: Average loss: 0.4815
Epoch: 3/10. Validation set: Average loss: 0.4094
Train: [0/1080 (0%)]	Loss: 0.384951
Epoch: 4/10. Train set: Average loss: 0.4150
Epoch: 4/10. Validation set: Average loss: 0.4150
Train: [0/1080 (0%)]	Loss: 0.385009
Epoch: 5/10. Train set: Average loss: 0.4055
Epoch: 5/10. Validation set: Average loss: 0.3703
Train: [0/1080 (0%)]	Loss: 0.332187
Epoch: 6/10. Train set: Average loss: 0.4325
Epoch: 6/10. Validation set: Average loss: 0.4115
Train: [0/1080 (0%)]	Loss: 0.353259
Epoch: 7/10. Train set: Average loss: 0.4608
Epoch: 7/10. Validation set: Average loss: 0.3817
Train: [0/1080 (0%)]	Loss: 0.496955
Epoch: 8/10. Train set: Average loss: 0.3890
Ep

In [65]:
# Define functions to extract embeddings using diferent datasets
def extract_embeddings(dataset, model, dims = 1024, opt = None):
    with torch.no_grad():
        model.eval()
        embeddings = np.zeros((len(dataset), dims))
        labels = list()
        for k, it_ in enumerate(dataset):
            images = it_[0].unsqueeze_(0)
            images = images.cuda()
            target = it_[1]
            if opt is None:
                aux = model.get_embedding(images).data.cpu().numpy()
            else:
                aux = model.get_embedding(images, opt).data.cpu().numpy()
            embeddings[k] = aux.reshape(1, dims)
            labels.append(target)
    return embeddings, labels


In [66]:
embeddings, labels = extract_embeddings(test_dataset, vgg_model, dims = 512)

In [67]:
#n_closest_images: (str), (int) ---> list
# Funcion que recibe la ruta de una imagen y de manera opcional un numero entero"n" (por defecto 5), que escribe 3 listas y entrega 1:
# Near_list: lista que contiene las n distancias mas cortas con respecto a la imagen entregada
# List_labels: lista que contiene las etiquetas correspondientes a las "n" imagenes con distancias mas cortas. Esta lista es entregada por la funcion.
# Label_ubication_list: lista que entrega la posicion de las "n" imagenes mas cercanas con respecto a su etiqueta.

def n_closest_images(image, n=5):
  img = Image.open(image).convert('RGB')
  img = trans(img)
  emb_img = vgg_model(img.unsqueeze(0).cuda())
  embedding_img = emb_img.data.cpu().numpy()
  list_dist, list_labels, neat_list, label_ubication_list = [], [], [], []
  for i in range(embeddings.shape[0]):
    emb2 = embeddings[i,:]
    dist = np.sum((emb2 - embedding_img)**2)
    list_dist.append(dist)
  neat_list[:] = list_dist
  neat_list.sort()
  for i in range(n):
    position = list_dist.index(neat_list[i])
    label_ubication = position - labels.index(labels[position])
    label_ubication_list.append(label_ubication)
    list_labels.append(labels[position])
  print (neat_list[:n], list_labels, label_ubication_list)
  return list_labels

In [68]:
n_closest_images("/content/drive/MyDrive/train/unclesam/unclesamrubberduckback400x400.jpg")[0]

[0.0, 0.060384943114031024, 0.10958801921959464, 0.1299177350882725, 0.1657605146418235] [tensor(281.), tensor(233.), tensor(279.), tensor(259.), tensor(233.)] [0, 0, 2, 1, 1]


tensor(281.)

In [69]:
# closest_image_after_rotation: (str) ---> list
# Funcion que recibe la ruta de una imagen, escribe 3 numeros y entrega 1:
# neat_list[:1]: Corresponde a la distancia mas corta entre la imagen entregada y una del dataset
# list_labels: Corresponde a la lista con las etiquetas de las imagenes del dataset, ordenadas desde las imagenes mas cercanas hasta las mas alejadas con respecto a la imagen entregada.
# esta lista es entregada por la funcion.
# Label_ubication_list: lista que entrega la posicion de las imagenes con respecto a su etiqueta. (Nota: si en la etiqueta tiene "n" imagenes, 0 representa la primera imagen y
# "k" representa la "k+1"-esima imagen)"
def closest_image_after_rotation(image):
  img = Image.open(image).convert('RGB')
  flip = transforms.RandomHorizontalFlip(p=1)
  img = flip(img)
  img = trans(img)
  emb_img = vgg_model(img.unsqueeze(0).cuda())
  embedding_img = emb_img.data.cpu().numpy()
  list_dist, list_labels, neat_list, label_ubication_list = [], [], [], []
  for i in range(embeddings.shape[0]):
    emb2 = embeddings[i,:]
    dist = np.sum((emb2 - embedding_img)**2)
    list_dist.append(dist)
  neat_list[:] = list_dist
  neat_list.sort()
  for i in range(1):
    position = list_dist.index(neat_list[i])
    label_ubication = position - labels.index(labels[position])
    label_ubication_list.append(label_ubication)
    list_labels.append(labels[position])
  print (neat_list[:1], list_labels, label_ubication_list)
  return list_labels

In [70]:
# percentage_check (dataset) ---> ()
# Funcion que recibe un dataset y rota las imagenes del mismo para luego pasarlas por la red y verificar cuantas imagenes son clasificadas correctamente en su etiqueta.
# El resultado es escrito como un porcentaje
def percentage_check(dataset):
  Right = 0
  for i in range(len(dataset.data)):
    img = dataset.data[i]
    label= closest_image_after_rotation(img)
    if label[0] == dataset.targets[i]:
      Right += 1 
  Total = len(dataset.targets)
  Percentage = 100.0*Right/Total
  print ("% " + str(Percentage) +" de las imagenes fueron clasificadas correctamente.")



In [73]:
percentage_check(test_dataset)

[0.014830298870920092] [tensor(228.)] [0]
[0.000790562431976653] [tensor(228.)] [1]
[0.0005347803744119539] [tensor(229.)] [0]
[0.0002853057729776891] [tensor(229.)] [1]
[0.005259248510410241] [tensor(229.)] [3]
[0.013021183815922278] [tensor(229.)] [3]
[0.0015105033794725058] [tensor(230.)] [0]
[0.0011089101533503347] [tensor(242.)] [1]
[0.0001676862191677187] [tensor(253.)] [0]
[0.0008023154293226153] [tensor(242.)] [1]
[0.0021497095523895873] [tensor(231.)] [0]
[0.026858760730506075] [tensor(238.)] [0]
[0.00762641719461795] [tensor(231.)] [2]
[7.534106779847337e-05] [tensor(232.)] [0]
[5.165396724945656e-09] [tensor(232.)] [1]
[0.0016120281162867835] [tensor(232.)] [2]
[2.583180997155305e-06] [tensor(232.)] [4]
[3.209828832736726e-06] [tensor(232.)] [3]
[0.005567425441621343] [tensor(233.)] [1]
[0.0011806258730058673] [tensor(233.)] [1]
[0.0016295508976897006] [tensor(235.)] [0]
[8.233695139201377e-05] [tensor(233.)] [3]
[0.00013855297733366426] [tensor(234.)] [0]
[0.011449900359762

In [72]:
test_dataset.class_to_idx

{'AlCapo': 0,
 'Alibaba': 1,
 'Americancop': 2,
 'Aquaman': 3,
 'Ariana': 4,
 'Artist': 5,
 'Astronaut': 6,
 'Avatara': 7,
 'Aztec': 8,
 'BabyBoyKeychain': 9,
 'Bali': 10,
 'Balifemale': 11,
 'BalletDancer': 12,
 'Balthazar': 13,
 'Basketball': 14,
 'Bat': 15,
 'Batmangreyholdy': 16,
 'BavarianMan': 17,
 'BavarianWoman': 18,
 'Bicycles': 19,
 'BigBen': 20,
 'Black': 21,
 'BlackSheep': 22,
 'BlackStar': 23,
 'Blacksheepmini': 24,
 'Blue': 25,
 'BlueSuede': 26,
 'Bluemini': 27,
 'BohemianQuacksody': 28,
 'Bombshell': 29,
 'Borntosun': 30,
 'Bowling': 31,
 'Boxer': 32,
 'Brit': 33,
 'Cactus': 34,
 'Caesar': 35,
 'Cannabis': 36,
 'CaptainQuack': 37,
 'Carnivalman': 38,
 'Carnivalwoman': 39,
 'Chef': 40,
 'Chewquacker': 41,
 'Chicken': 42,
 'ChimneySweep': 43,
 'Christmasjumper': 44,
 'Cleaningfairy': 45,
 'Clouds': 46,
 'Cop': 47,
 'Crocodile': 48,
 'Cyber': 49,
 'Cybersoldier': 50,
 'Dali': 51,
 'Dark': 52,
 'DarkDuckWoman': 53,
 'Delftblue': 54,
 'Diamont': 55,
 'Dino': 56,
 'Diva': 57,
